In [91]:
import folium

In [92]:
from folium import plugins

In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [94]:
travel_index = pd.read_csv('hafas_vbb.csv', index_col=0)
travel_index

,latitude,longitude,min travel time
0,52.502793,13.473981,904.584445
1,52.502793,13.467363,636.986839
2,52.502793,13.460744,859.758908
3,52.502793,13.454125,846.968438
4,52.502793,13.447507,499.271294
5,52.508708,13.473981,1179.492022
6,52.508708,13.467363,807.374833
7,52.508708,13.460744,813.429644
8,52.508708,13.454125,976.752917
9,52.508708,13.447507,859.535770


In [102]:
m = folium.Map((52.515998, 13.457574), zoom_start=14)
for index, row in travel_index.iterrows():
    mtt = row['min travel time']
    if mtt < 600:
        color = 'green'
    elif mtt < 900:
        color = 'yellow'
    else:
        color = 'red'
    folium.CircleMarker((row['latitude'], row['longitude']), fill_color=color, opacity=1.0, color=color, stroke=True, tooltip=mtt).add_to(m)

In [103]:
m